In [97]:
import dlib
import datetime
import os
import numpy as np
from PIL import Image

In [44]:
options = dlib.simple_object_detector_training_options()
#opcoes.add_left_right_image_flips = True
options.num_threads = 8
options.be_verbose = True
options.C = 5

In [45]:
path_train = r'D:\Teles\Desenvolvimento\JupyterNotebooks\DLib\images\doriana\train.xml'
#path_train = r'D:\Teles\Desenvolvimento\JupyterNotebooks\DLib\images\doriana\output\train.xml'
path_model = r'D:\Teles\Desenvolvimento\JupyterNotebooks\DLib\models\doriana.svm'

In [46]:
t1 = datetime.datetime.now()
dlib.train_simple_object_detector(path_train, path_model, options)
t2 = datetime.datetime.now()
print(t2-t1)

0:00:30.548211


In [81]:
def treinar(path_xml, path_model, label):
    
    model = os.path.join(path_model, label + '.svm')
    
    options = dlib.simple_object_detector_training_options()
    options.num_threads = 8
    options.be_verbose = False
    options.C = 1
    
    t1 = datetime.datetime.now()
    dlib.train_simple_object_detector(path_xml, model, options)
    t2 = datetime.datetime.now()
    print(t2-t1)
    
    

In [107]:
def preparar_deteccao(path_model, labels):
    detectores = []
    classes = ['1','2','3','4','5','6','7','8','9']
    for label in labels:
        model = os.path.join(path_model, label + '.svm')
        detectores.append(dlib.fhog_object_detector(model))
    return detectores, classes

In [108]:
labels = ['um','dois','tres','quatro','cinco', 'seis', 'sete', 'oito', 'nove']
path = r'D:\MachineLearning\dataset\Teles\Numeros'
detectores, classes = preparar_deteccao(path, labels)

D:\MachineLearning\dataset\Teles\Numeros\um.svm
D:\MachineLearning\dataset\Teles\Numeros\dois.svm
D:\MachineLearning\dataset\Teles\Numeros\tres.svm
D:\MachineLearning\dataset\Teles\Numeros\quatro.svm
D:\MachineLearning\dataset\Teles\Numeros\cinco.svm
D:\MachineLearning\dataset\Teles\Numeros\seis.svm
D:\MachineLearning\dataset\Teles\Numeros\sete.svm
D:\MachineLearning\dataset\Teles\Numeros\oito.svm
D:\MachineLearning\dataset\Teles\Numeros\nove.svm


In [109]:
def detectar(imagem, detectores, classes):
    img = np.array(Image.open(imagem))
    [objects, confidences, detector_idxs] = dlib.fhog_object_detector.run_multiple(detectores, 
                                                                                   img,
                                                                                   upsample_num_times=2,
                                                                                   adjust_threshold=0.0)
    print('{} => detectados: {}'.format(imagem, len(objects)))     

In [111]:
img = r'D:\MachineLearning\dataset\Teles\Numeros\cinco01.jpg'
detectar(img, detectores, classes)

D:\MachineLearning\dataset\Teles\Numeros\cinco01.jpg => detectados: 0


In [84]:
labels = ['um','dois','tres','quatro','cinco', 'seis', 'sete', 'oito', 'nove']
path = r'D:\MachineLearning\dataset\Teles\Numeros'
for label in labels:
    config = os.path.join(path, label +'.xml')
    print(config)
    treinar(config, path, label)
print('fim')

D:\MachineLearning\dataset\Teles\Numeros\um.xml
0:00:15.198011
D:\MachineLearning\dataset\Teles\Numeros\dois.xml
0:00:26.894987
D:\MachineLearning\dataset\Teles\Numeros\tres.xml
0:00:18.233995
D:\MachineLearning\dataset\Teles\Numeros\quatro.xml
0:00:24.223031
D:\MachineLearning\dataset\Teles\Numeros\cinco.xml
0:00:15.524963
D:\MachineLearning\dataset\Teles\Numeros\seis.xml
0:00:33.137998
D:\MachineLearning\dataset\Teles\Numeros\sete.xml
0:00:07.615000
D:\MachineLearning\dataset\Teles\Numeros\oito.xml
0:00:25.448000
D:\MachineLearning\dataset\Teles\Numeros\nove.xml
0:00:19.899992
fim


<b>Converter Pascal XML para formato DLib</b>

In [74]:
import os
from xml.dom import minidom

In [75]:
docXml = r'D:\MachineLearning\dataset\Teles\Numeros\sudoku.xml'
docImage = r'D:\MachineLearning\dataset\Teles\Numeros\sudoku.jpg'
path = r'D:\MachineLearning\dataset\Teles\Numeros'
mydoc = minidom.parse(docXml)

In [76]:
itens = mydoc.getElementsByTagName('object')

In [77]:
dados = []
for item in itens:
    nome = item.getElementsByTagName('name')[0].firstChild.nodeValue
    x1 = item.getElementsByTagName('xmin')[0].firstChild.nodeValue
    y1 = item.getElementsByTagName('ymin')[0].firstChild.nodeValue
    x2 = item.getElementsByTagName('xmax')[0].firstChild.nodeValue
    y2 = item.getElementsByTagName('ymax')[0].firstChild.nodeValue
    dados.append([nome, x1, y1, x2, y2])

In [78]:
xml1 = ("<?xml version='1.0' encoding='ISO-8859-1'?>" +
     "<?xml-stylesheet type='text/xsl' href='image_metadata_stylesheet.xsl'?>" +
     "<dataset>" +
        "<images>")
xml2 = "</images></dataset>"

In [79]:
def gravar(txt, label):
    label += '.xml'
    arq = os.path.join(path, label)
    with open(arq, 'w') as f:
        f.write(txt)

In [80]:
labels = ['um','dois','tres','quatro','cinco', 'seis', 'sete', 'oito', 'nove']
for label in labels:
    boxes = [d for d in dados if d[0] == label]
    txt = ''
    for box in boxes:
        label, x1, y1, x2, y2 = box
        txt += "<box top='{}' left='{}' width='{}' height='{}'/>".format(x1, y1, int(y2)-int(y1)+1, int(x2)-int(x1)+1)
    txt = "<image file='{}'>{}</image>".format(docImage, txt)
    arquivo = xml1 + txt + xml2
    gravar(arquivo, label)
print('fim')
        


fim
